# Demo

Injection of first order logic rules into a neural network for iris classification task.

Download antlr4 jar and create parser for Datalog.g4 grammar.

In [1]:
! wget https://www.antlr.org/download/antlr-4.9.2-complete.jar
! export CLASSPATH="./antlr-4.9.2-complete.jar:$CLASSPATH"
! java -jar ./antlr-4.9.2-complete.jar -Dlanguage=Python3 resources/Datalog.g4 -visitor -o resources/dist
! rm ./antlr-4.9.2-complete.jar

--2022-03-07 12:26:22--  https://www.antlr.org/download/antlr-4.9.2-complete.jar
Resolving www.antlr.org (www.antlr.org)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to www.antlr.org (www.antlr.org)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100564 (2.0M) [application/java-archive]
Saving to: ‘antlr-4.9.2-complete.jar’

antlr-4.9.2-complet 100%[===================>]   2.00M  6.99MB/s    in 0.3s    

2022-03-07 12:26:23 (6.99 MB/s) - ‘antlr-4.9.2-complete.jar’ saved [2100564/2100564]



Some import.

In [2]:
from antlr4 import InputStream, CommonTokenStream
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.random_seed import set_random_seed
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from psyki.logic.datalog.grammar.adapters import Antlr4
from resources.dist.resources.DatalogLexer import DatalogLexer
from resources.dist.resources.DatalogParser import DatalogParser
from psyki.ski.injectors import LambdaLayer
from test.utils import get_mlp
from test.resources.rules import get_rules

Loading iris dataset and separation into train and test set.

In [3]:
x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)

train, test = train_test_split(dataset, test_size=0.5, random_state=0)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


Import FOL rules:

- PL <= 2.28 <- X = setosa
- PL > 2.28 ^ PW > 1.64 <- X = virginica
- PL > 2.28 ^ PW <= 1.64 <- X = versicolor

In [4]:
features_mapping = {
    'SL': 0,
    'SW': 1,
    'PL': 2,
    'PW': 3,
}
class_mapping = {
    'setosa': 0,
    'virginica': 1,
    'versicolor': 2
}

iris_rules = get_rules('iris')
formulae = [Antlr4().get_formula(DatalogParser(CommonTokenStream(DatalogLexer(InputStream(rule)))).formula()) for rule in iris_rules]

Injection of fuzzy logic function derived from FOL rules into a neural network.

In [5]:
set_random_seed(0)
input_features = Input((4,), name='Input')
network = get_mlp(input_layer=input_features, output=3, layers=3, neurons=16, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = LambdaLayer(model, class_mapping, features_mapping)
injector.inject(formulae)
injector.predictor.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
injector.predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           80          Input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           272         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 3)            51          dense_1[0][0]                    
____________________________________________________________________________________________

Training.

In [6]:
train_x, train_y = train.iloc[:,:-1], train.iloc[:,-1]
injector.predictor.fit(train_x, train_y, verbose=1, batch_size=4, epochs=10)

Epoch 1/10


2022-03-07 12:26:25.183937: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-07 12:26:25.184095: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


19/19 [==============================] - 0s 574us/step - loss: 1.0992 - accuracy: 0.6133
Epoch 2/10
19/19 [==============================] - 0s 514us/step - loss: 0.8600 - accuracy: 0.6267
Epoch 3/10
19/19 [==============================] - 0s 479us/step - loss: 0.6884 - accuracy: 0.6267
Epoch 4/10
19/19 [==============================] - 0s 489us/step - loss: 0.5835 - accuracy: 0.6400
Epoch 5/10
19/19 [==============================] - 0s 474us/step - loss: 0.5261 - accuracy: 0.6400
Epoch 6/10
19/19 [==============================] - 0s 489us/step - loss: 0.4942 - accuracy: 0.6800
Epoch 7/10
19/19 [==============================] - 0s 471us/step - loss: 0.4750 - accuracy: 0.6800
Epoch 8/10
19/19 [==============================] - 0s 460us/step - loss: 0.4634 - accuracy: 0.6800
Epoch 9/10
19/19 [==============================] - 0s 484us/step - loss: 0.4521 - accuracy: 0.6667
Epoch 10/10
19/19 [==============================] - 0s 464us/step - loss: 0.4419 - accuracy: 0.6933


Removing the injected rules from the network.

In [7]:
new_network = Model(injector.predictor.input, injector.predictor.layers[-3].output)
new_network.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_network.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


Evaluation.

In [8]:
test_x, test_y = test.iloc[:,:-1], test.iloc[:,-1]
new_network.evaluate(test_x, test_y)

3/3 [==============================] - 0s 739us/step - loss: 0.2788 - accuracy: 0.9867


[0.278796523809433, 0.9866666793823242]